In [3]:
import pandas as pd
import numpy as np
import os

folder_path = 'playlists'

In [ ]:
## function which takes in a list of playlist csv files and outputs a dataframe with each playlist as a row and songs as columns
def make_matrix_song():
    df = pd.DataFrame()
    df['Playlist Name'] = np.nan
    songs_set = set()

    # List all CSV files in the folder
    row_number = 0
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            playlist_df = pd.read_csv(file_path)
            df.loc[row_number, 'Playlist Name'] = os.path.splitext(filename)[0]

            for _, row in playlist_df.iterrows():
                song = row['Track Name']
                if song not in songs_set:
                    # Add the song to the set
                    songs_set.add(song)
                    # Add a new column for the song in the DataFrame
                    df[song] = np.nan

                df.loc[row_number, song] = 1
            row_number += 1

    return df

songs = make_matrix_song()
songs.to_csv("song_data.csv", index=False)

In [42]:
## CORRECT VERSION. USE ME. PICK ME.

def make_matrix2():
    # Initialize a DataFrame for the final output
    final_df = pd.DataFrame()

    # Set for storing unique artist names
    all_artists = set()

    # List all CSV files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            playlist_df = pd.read_csv(file_path)
            playlist_name = os.path.splitext(filename)[0]

            # Dictionary to store artist counts for the current playlist
            artist_count = {}

            for _, row in playlist_df.iterrows():
                # Extract and process artists
                artists = [artist.strip() for artist in row['Artist Name(s)'].split(',')]
                for artist in artists:
                    all_artists.add(artist)
                    artist_count[artist] = artist_count.get(artist, 0) + 1

            # Create a DataFrame for the current playlist
            playlist_artist_df = pd.DataFrame([{'Playlist Name': playlist_name, **artist_count}])
            final_df = pd.concat([final_df, playlist_artist_df], ignore_index=True)

    # Add columns for any missing artists in each playlist
    missing_artists = all_artists - set(final_df.columns)
    for artist in missing_artists:
        final_df[artist] = 0

    # DONT Fill NaN with 0s
    #final_df.fillna(0, inplace=True)

    return final_df

df = make_matrix2()
df.to_csv("artists_ORIGINAL2.csv", index=False)

In [41]:
#NORMALIZE
# Calculate the norm of the array --- subtract

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = df.iloc[:, 1:]
scaler = MinMaxScaler()

# Create a mask for NaN values
nan_mask = df.isna()

# Apply scaler to non-NaN values
scaled_values = df.copy()
scaled_values[~nan_mask] = scaler.fit_transform(df[~nan_mask])

lower_bound = -10
upper_bound = 10
scaled_values[~nan_mask] = lower_bound + (scaled_values[~nan_mask] - scaled_values[~nan_mask].min(axis=0)) * (upper_bound - lower_bound) / (scaled_values[~nan_mask].max(axis=0) - scaled_values[~nan_mask].min(axis=0))

scaled_dataframe = pd.DataFrame(scaled_values, columns=df.columns)
print(scaled_dataframe)


    Jhayco  Bad Bunny  Rauw Alejandro  Alejo  Feid  ROBI  Alvaro Diaz  \
0     10.0  10.000000    1.000000e+01   10.0  10.0   NaN          NaN   
1      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
2      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
3      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
4      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
5      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
6      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
7      NaN -10.000000   -1.000000e+01    NaN   NaN   NaN          NaN   
8    -10.0  -9.545455             NaN    NaN   NaN   NaN          NaN   
9      NaN        NaN             NaN    NaN   NaN   NaN          NaN   
10     NaN        NaN             NaN    NaN   NaN   NaN          NaN   
11     NaN        NaN             NaN    NaN   NaN   NaN          NaN   
12     NaN        NaN             NaN    NaN   NaN 

In [43]:
from src.alternative_minimization import matrix_completion

df = df.apply(pd.to_numeric, errors='coerce')
df = np.array(df)
new_df = matrix_completion(df, 30, 200)
print(new_df)
new_df = pd.DataFrame(new_df)
new_df.to_csv("test_data.csv", index=False)


/Users/rheamalhotra/Desktop/music/src/alternative_minimization.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  V[i, :] = np.linalg.lstsq(U_omega, y_omega)[0]


[[1. 5. 5. ... 1. 1. 1.]
 [5. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 5. ... 1. 1. 1.]]


In [44]:
## RECOMMEND
max_values_by_row = new_df.max(axis=1)
max_values_by_column = new_df.max(axis=0)

print("Maximum values by row:")
print(max_values_by_row)
print("Maximum values by column:")
print(max_values_by_column)

Maximum values by row:
0     5.0
1     5.0
2     2.0
3     5.0
4     5.0
5     5.0
6     5.0
7     5.0
8     5.0
9     5.0
10    5.0
11    5.0
12    5.0
13    5.0
14    5.0
15    2.0
16    5.0
17    5.0
dtype: float64
Maximum values by column:
0      5.0
1      5.0
2      5.0
3      5.0
4      5.0
      ... 
642    1.0
643    1.0
644    1.0
645    1.0
646    1.0
Length: 647, dtype: float64


In [1]:
## RUN SVD ON ARTISTS dataframe

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from matplotlib import pyplot as plt
import csv
from sklearn.decomposition import PCA
import seaborn as sns
sns.set(style='ticks')

pcaArtists = PCA(n_components=6)
pcaArtists.fit(dataQsOnly.loc[dataQsOnly["country"]==1])